In [ ]:
#%%capture

#!pip install pafy youtube-dl moviepy tensorflow pydot
#!pip install --upgrade pip

In [ ]:
import os
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model


In [ ]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)


In [ ]:
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF50.rar

In [ ]:
plt.figure(figsize=(20, 20))
all_classes_names = os.listdir('UCF50')

random_range = random.sample(range(len(all_classes_names)), 20)

for counter, random_index in enumerate(random_range, 1):
    selected_class_Name = all_classes_names[random_index]
    video_files_names_list = os.listdir(f'UCF50/{selected_class_Name}')
    selected_video_file_name = random.choice(video_files_names_list)
    video_reader = cv2.VideoCapture(f'UCF50/{selected_class_Name}/{selected_video_file_name}')
    _, bgr_frame = video_reader.read()
    video_reader.release()
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)
    cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    plt.subplot(5, 4, counter);plt.imshow(rgb_frame);plt.axis('off')




In [ ]:
# preprocess the dataset

IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 20
DATASET_DIR = 'UCF50'
CLASSES_LIST = ["WalkingWithDog", "TaiChi", "Swing", "HorseRace", "JumpingJack"]



In [ ]:
# extract, resize and normalize frames

def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_Frame = resized_frame / 255
        frames_list.append(normalized_Frame)
    video_reader.release()

    return frames_list



In [ ]:
def create_dataset():
    features =[]
    labels =[]
    video_files_paths = []

    for class_index, class_name in enumerate(CLASSES_LIST):
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            frames = frames_extraction(video_file_path)
            if len(frames)==SEQUENCE_LENGTH:
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
    features = np.asarray(features)
    labels = np.array(labels)

    return features, labels, video_file_path

In [ ]:
features, labels, video_files_paths = create_dataset()

In [ ]:

one_hot_encoded_labels = to_categorical(labels)


In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.25, shuffle=True, random_state= seed_constant)


In [ ]:
print (type(features_train))
print (type(labels_train))
#print (features_train)
print (labels_train)


In [ ]:
def create_convlstm_model():
    model=Sequential()
    model.add(ConvLSTM2D(filters=4, kernel_size = (3,3), activation = 'tanh', data_format = "channels_last", recurrent_dropout=0.2, return_sequences=True, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=8, kernel_size = (3,3), activation = 'tanh', data_format = "channels_last", recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=14, kernel_size = (3,3), activation = 'tanh', data_format = "channels_last", recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=16, kernel_size = (3,3), activation = 'tanh', data_format = "channels_last", recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))

    model.add(Flatten())
    model.add(Dense(len(CLASSES_LIST), activation = "softmax"))
    model.summary()
    return model


In [ ]:
convlstm_model = create_convlstm_model()


In [ ]:
plot_model(convlstm_model, to_file='convlstm_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 100, mode = 'min', restore_best_weights= True)
convlstm_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs=50, batch_size = 4, shuffle = True, validation_split=0.2, callbacks=[early_stopping_callback])


In [ ]:
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)

In [ ]:
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

date_time_format = "%Y_%m_%d__%H%_M_%S"
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
model_file_name = f"convlstm_model_dt_{current_date_time_string}_loss_{model_evaluation_loss}_acc_{model_evaluation_accuracy}.h5"
convlstm_model.save(model_file_name)



In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]
    epochs = range(len(metric_value_1))
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    plt.title(str(plot_name))
    plt.legend()


In [ ]:
plot_metric(convlstm_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
plot_metric(convlstm_model_training_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')

In [ ]:
def create_LRCN_model():
    model=Sequential()
    model.add(TimeDistributed(Conv2D(16, (3,3), padding='same', activation = 'relu'), input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH,3)))
    model.add(TimeDistributed(MaxPooling2D((4,4))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4,4))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4,4))))
    #model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Flatten()))

    model.add(LSTM(32))
    model.add(Dense(len(CLASSES_LIST), activation='softmax'))
    model.summary()

    return model

In [ ]:
LRCN_model = create_LRCN_model()

In [ ]:
plot_model(LRCN_model, to_file='LRCN_plot.png', show_shapes=True, show_layer_names = True)

In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 100, mode = 'min', restore_best_weights= True)
LRCN_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs=50, batch_size = 4, shuffle = True, validation_split=0.2, callbacks=[early_stopping_callback])


In [ ]:
model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)

In [ ]:
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

date_time_format = "%Y_%m_%d__%H%_M_%S"
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
model_file_name = f"LRCN_model_dt_{current_date_time_string}_loss_{model_evaluation_loss}_acc_{model_evaluation_accuracy}.h5"
LRCN_model.save(model_file_name)

In [ ]:
plot_metric(LRCN_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
plot_metric(LRCN_model_training_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')

In [ ]:
def download_youtube_video(youtube_video_url, output_directory):
    print ()
    video = pafy.new(youtube_video_url)
    title = video.title
    print ('video title', title)
    video_best = video.getbest()
    output_file_path = f'{output_directory}/{title}.mp4'
    video_best.download(filepath = output_file_path, quite=True)
    return title

In [ ]:
test_video_directory = 'test_video'
os.makedirs(test_video_directory, exist_ok = True)
video_title = download_youtube_video('https://www.youtube.com/watch?v=8u0qjmHIOcE', test_video_directory)
input_video_file_path = f'{test_video_directory}/{video_title}.mp4'

In [ ]:
def predict_on_video(video_file_path, output_file_path, SEQUENCE_LENGTH, model_type):

    if model_type == 'LRCN':
        model = load_model('LRCN_model.h5')
    else:
        model = load_model('convlstm_model.h5')
    
    video_reader = cv2.VideoCapture(video_file_path)
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', "P", "4", "V"), video_reader.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))
    frames_queue = deque(maxlen = SEQUENCE_LENGTH)

    predicted_class_name = ''
    while video_reader.isOpened():
        ok, frame = video_reader.read()
        if not ok:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame /255
        frames_queue.append(normalized_frame)

        if len(frames_queue) == SEQUENCE_LENGTH:
            predicted_labels_probabilities = model.predict(np.expand_dims(frames_queue, axis=0))[0]
            predicted_label = np.argmax(predicted_labels_probabilities)
            predicted_class_name = CLASSES_LIST[predicted_label]

        cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        video_writer.write(frame)
        
    video_reader.release()
    video_writer.release()



In [ ]:
video_title = 'test_video'
input_video_file_path = f'{test_video_directory}/{video_title}.mp4'
output_video_file_path = f'{test_video_directory}/{video_title}-Output-SeqLen{SEQUENCE_LENGTH}.mp4'
model_type = 'LRCN'
#model_type = 'CONVLSTM'
predict_on_video(input_video_file_path, output_video_file_path, SEQUENCE_LENGTH, model_type)
VideoFileClip(output_video_file_path, audio=False, target_resolution=(300, None)).ipython_display()